In [30]:
#Importing necessary libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
import warnings
warnings.filterwarnings('ignore')
from geopy.geocoders import Nominatim

In [31]:
# Nominatim refers to the geocoding that generates location 
geolocator = Nominatim(user_agent = 'Suman')
location = geolocator.geocode("Upstate Medical University")
print(location.point) #point gives latitude and longitude values
print(location.address)

43 2m 26.9462s N, 76 8m 20.9173s W
State University of New York Upstate Medical University, 750, East Adams Street, University Hill, Syracuse, Onondaga County, New York, 13210, United States


In [32]:
point = location.point
print("latitude:",point.latitude)
print("longitude:",point.longitude)

latitude: 43.0408184
longitude: -76.1391437


In [33]:
universities = pd.read_csv("/Users/sdevkota01/Downloads/Python/GEO/top_universities.csv")
universities.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


In [34]:
# using lambda function to apply geocoder to universities
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return None
    
universities[['Latitude','Longitude']] = universities.apply( lambda x: my_geocoder(x['Name']),axis =1)


In [35]:
universities.head()

,Name,Latitude,Longitude
0,University of Oxford,51.758708,-1.255668
1,University of Cambridge,52.199852,0.119739
2,Imperial College London,51.498959,-0.175641
3,ETH Zurich,47.408327,8.507564
4,UCL,51.523581,-0.132977


In [36]:
# drop universities that were not sucessfully geocoded
universities = universities.loc[~np.isnan(universities['Latitude'])]
#adding geometry column representating point from latitude and longitude
universities = gpd.GeoDataFrame( universities,geometry=gpd.points_from_xy(universities.Longitude,universities.Latitude))
#giving crs value to universities
universities.crs = {'init':'epsg:4326'}


In [37]:
universities.head()

,Name,Latitude,Longitude,geometry
0,University of Oxford,51.758708,-1.255668,POINT (-1.25567 51.75871)
1,University of Cambridge,52.199852,0.119739,POINT (0.11974 52.19985)
2,Imperial College London,51.498959,-0.175641,POINT (-0.17564 51.49896)
3,ETH Zurich,47.408327,8.507564,POINT (8.50756 47.40833)
4,UCL,51.523581,-0.132977,POINT (-0.13298 51.52358)


In [38]:
#creating a map
Map = folium.Map(location=[54,15],tiles = 'openstreetmap',zoom_start = 2)
Map

In [40]:
#adding universities location to the map
for idx,row in universities.iterrows():
    Marker([row['Latitude'],row['Longitude']],popup = row['Name']).add_to(Map)
#display Map
Map

In [29]:
#looks like the value that geocoder has given in not all accurate since some of the location are not in europe